## Consegna
Si richiede un'implementazione di un esercizio di Topic Modeling, utilizzando librerie open (come ad es. GenSim (https://radimrehurek.com/gensim/). Si richiede l'utilizzo di un corpus di almeno 1k documenti. Testare un algoritmo (ad esempio LDA) con più valori di k (num. di topics) e valutare la coerenza dei risultati, attraverso fine-tuning su parametri e pre-processing. Update: essendo che spesso i topic, per essere interpretabili, devono contenere content words, potete pensare di filtrare solamente i sostantivi in fase di preprocessing (cioè POS=noun).

## Import

In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

## Caricamento Corpus 

In [ ]:
# Carica il corpus 20 Newsgroups
newsgroups = fetch_20newsgroups(
    subset='all', remove=('headers', 'footers', 'quotes'))


## Estrazione sostantivi e pulizia del corpus

In [ ]:
# Estrai i sostantivi dai documenti e crea il corpus
stop_words = set(stopwords.words('english'))
corpus = []
for document in newsgroups.data:
    words = word_tokenize(document)
    words = [word.lower() for word in words if word.isalpha()
             and word.lower() not in stop_words]
    tagged_words = pos_tag(words)
    nouns = [word for word, pos in tagged_words if pos == 'NN']
    corpus.append(nouns)
    print(corpus)


## Creazione del dizionario e del corpus in formato bag-of-words

In [ ]:

dictionary = Dictionary(corpus)
corpus_bow = [dictionary.doc2bow(doc) for doc in corpus]


## Test di diversi valori di k (num. di topics)

In [ ]:

k_values = [5, 10, 15, 20, 25, 30, 40, 50, 75, 100, 150]
coherence_scores = []

for k in k_values:
    lda_model = LdaModel(corpus_bow, num_topics=k, id2word=dictionary)
    coherence_model = CoherenceModel(
        model=lda_model, texts=corpus, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append(coherence_score)
    print(f"Number of topics: {k}, Coherence Score: {coherence_score}")


## Stampa dei risultati

In [ ]:
# Plot dei risultati
plt.plot(k_values, coherence_scores, marker='o')
plt.xlabel('Number of Topics')
plt.ylabel('Coherence Score')
plt.title('Coherence Score vs. Number of Topics')
plt.show()
